# Tutorial 4: Multi-level column indices (`MultiIndex`)

A MultiIndex is an index with multiple, hierarchical levels. We use a multiindex for the column headers of a 
dataframe. This allows each column to have multiple keys associated with it--one key for each level. 

The multiindex is a native part of the `pandas` package. For more documentation, see their [MultiIndex / advanced indexing page](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

MultiIndexes make it easy to associate multiple identifiers with a single column.  To show what this looks like we'll look at phosphoproteomics data in the endometrial cancer dataset.

In [1]:
# Throughout the tutorial we will be using the endometrial and colon datasets
import cptac
cptac.download(dataset="endometrial", version="latest")
cptac.download(dataset="colon", version="latest")

True

In [2]:
en = cptac.Endometrial()
phospho = en.get_phosphoproteomics()
phospho.head()

Name         AAAS                  AACS  AAED1 AAGAB          AAK1         \
Site         S495    S541   Y485   S618    S12  S310   S311    S14    S18   
Patient_ID                                                                  
C3L-00006     NaN     NaN    NaN -0.881 -1.810   NaN    NaN    NaN -0.242   
C3L-00008     NaN     NaN    NaN    NaN  0.084   NaN    NaN -1.110 -0.383   
C3L-00032  -0.202     NaN    NaN    NaN -1.880   NaN    NaN    NaN  0.382   
C3L-00090  -0.002     NaN -0.407    NaN    NaN   NaN    NaN    NaN    NaN   
C3L-00098   0.556 -0.0461    NaN    NaN  0.941   NaN  0.429  0.362  0.697   

Name                ...     ZZZ3                                            \
Site           S20  ...     S397 S411 S420   S424     S426 S468   S89 T415   
Patient_ID          ...                                                      
C3L-00006  -0.2420  ...  0.18400  NaN  NaN    NaN -0.20500  NaN   NaN  NaN   
C3L-00008  -1.0900  ... -0.17100  NaN  NaN -0.393 -0.17100  NaN  0.29  NaN   
C3L-00032  -0.0416  ...      NaN  NaN  NaN    NaN      NaN  NaN   NaN  NaN   
C3L-00090  -0.5550  ...  0.13970  NaN  NaN    NaN -0.55900  NaN   NaN  NaN   
C3L-00098  -0.0529  ... -0.15875  NaN  NaN  0.196  0.06175  NaN   NaN  NaN   

Name                        
Site          T418    Y399  
Patient_ID                  
C3L-00006      NaN     NaN  
C3L-00008   0.1605 -0.0635  
C3L-00032      NaN     NaN  
C3L-00090      NaN  0.2980  
C3L-00098      NaN -0.2900  

[5 rows x 73212 columns]

The first level of keys is the name of the gene associated with the column of data. The second level is the site of acetylation. Each column has its own Name and Site associated with it. 
Normally, as above, keys that repeat for adjacent columns are not displayed, to make it easier to read the dataframe. For example, in the table above the first 7 columns are all for the A2M gene, but the label is only shown for the first column. 

# Join functions with multiindices 
The join functions have been written to handle multiindices. More information on the join functions can be found in the joining_dataframes tutorial. 
An example of joining a multiindexed dataframe (in this case phosphoproteomics) with a non multiindexed dataframe (in this case CNV) is below. 

In [3]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics")
phospho_and_CNV.head()

cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-00006.N, C3L-00361.N, C3L-00563.N, C3L-00586.N, C3L-00601.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00932.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01282.N, C3L-01304.N, C3L-01307.N, C3L-01311.N, C3L-01744.N, C3N-00200.N, C3N-00333.N, C3N-00383.N, C3N-00729.N, C3N-00858.N, C3N-00866.N, C3N-01211.N, C3N-01346.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (<ipython-input-3-33a764ee6b29>, line 1)


Name       A1BG_CNV A1BG-AS1_CNV A1CF_CNV A2M_CNV A2M-AS1_CNV A2ML1_CNV  \
Site                                                                      
Patient_ID                                                                
C3L-00006      0.00         0.00    -0.01    0.00        0.00      0.00   
C3L-00008      0.01         0.01     0.00    0.00        0.00      0.00   
C3L-00032      0.01         0.01     0.00    0.00        0.00      0.00   
C3L-00090     -0.04        -0.04     0.36   -0.05       -0.05     -0.05   
C3L-00098      0.85         0.85     0.16   -0.41       -0.41     -0.41   

Name       A2MP1_CNV A3GALT2_CNV A4GALT_CNV A4GNT_CNV  ...  \
Site                                                   ...   
Patient_ID                                             ...   
C3L-00006       0.00        0.00       0.01     -0.01  ...   
C3L-00008       0.00        0.00       0.01      0.00  ...   
C3L-00032       0.00        0.01       0.00      0.00  ...   
C3L-00090      -0.05       -0.04      -0.04     -0.05  ...   
C3L-00098      -0.41       -0.12       0.15      0.19  ...   

Name       ZZZ3_phosphoproteomics                                            \
Site                         S397 S411 S420   S424     S426 S468   S89 T415   
Patient_ID                                                                    
C3L-00006                 0.18400  NaN  NaN    NaN -0.20500  NaN   NaN  NaN   
C3L-00008                -0.17100  NaN  NaN -0.393 -0.17100  NaN  0.29  NaN   
C3L-00032                     NaN  NaN  NaN    NaN      NaN  NaN   NaN  NaN   
C3L-00090                 0.13970  NaN  NaN    NaN -0.55900  NaN   NaN  NaN   
C3L-00098                -0.15875  NaN  NaN  0.196  0.06175  NaN   NaN  NaN   

Name                        
Site          T418    Y399  
Patient_ID                  
C3L-00006      NaN     NaN  
C3L-00008   0.1605 -0.0635  
C3L-00032      NaN     NaN  
C3L-00090      NaN  0.2980  
C3L-00098      NaN -0.2900  

[5 rows x 101269 columns]

Since the CNV dataframe doesn't have the level "Site", it is filled in with NANs, so that it can be joined to the phosphoproteomics dataframe. 

# How to select from multiindex


## Selecting based on all levels
We can select single columns by passing the proper keys for all levels of the multiindex. For example, to get the acetylproteomics for A2M at site K1176, we'd do the following:

In [4]:
acetyl = en.get_acetylproteomics()
all_levels_selection = acetyl["A2M"]["K1176"]
all_levels_selection.head(10)

Patient_ID
C3L-00006    1.080
C3L-00008    0.477
C3L-00032      NaN
C3L-00090   -0.608
C3L-00098    1.630
C3L-00136   -0.210
C3L-00137   -0.175
C3L-00139   -0.223
C3L-00143   -1.120
C3L-00145      NaN
Name: K1176, dtype: float64

## Selecting based on one level
We can easily select multiple columns from our multiindex dataframe, based on just the "Name" level of the multiindex:

In [5]:
gene1_filter = acetyl.columns.get_level_values("Name").str.startswith("AA") # Select all columns where the gene starts with "AA". This will grab every column where the key "Name" starts with AA
gene1_data = acetyl.loc[:, gene1_filter]
gene1_data.head()

Name       AACS  AAGAB     AAK1 AARS AASDH AASS               AATF    
Site       K391   K290     K201 K338  K765 K138 K539    K93   K380 K74
Patient_ID                                                            
C3L-00006   NaN  0.461      NaN  NaN   NaN  NaN  NaN    NaN  1.300 NaN
C3L-00008   NaN  1.770      NaN  NaN   NaN  NaN  NaN    NaN  0.652 NaN
C3L-00032   NaN -0.815 -0.00573  NaN   NaN  NaN  NaN -0.305  0.398 NaN
C3L-00090   NaN    NaN      NaN  NaN   NaN  NaN  NaN    NaN    NaN NaN
C3L-00098   NaN  0.205      NaN  NaN   NaN  NaN  NaN    NaN  0.244 NaN

## Selecting based on a different level of the multiindex
We can also select based on one of the inner levels of the multiindex. For example, to get data for all tyrosine phosphorylation sites:

In [6]:
y_site_filter = phospho.columns.get_level_values("Site").str.contains("Y") # Create a boolean filter selecting all columns where the Site level contains a "Y"

y_sites = phospho.loc[:, y_site_filter] # Select the columns
y_sites.head()

Name         AAAS    AAK1   ABCC1   ABI1 ABI2 ABI3   ABL1   ABL2 ABLIM1  \
Site         Y485    Y687    Y920   Y213 Y213 Y361   Y204    Y80   Y357   
Patient_ID                                                                
C3L-00006     NaN     NaN  0.4340 -0.591  NaN  NaN -0.759    NaN    NaN   
C3L-00008     NaN     NaN -0.5110    NaN  NaN  NaN -1.210  0.321    NaN   
C3L-00032     NaN     NaN     NaN    NaN  NaN  NaN    NaN    NaN    NaN   
C3L-00090  -0.407     NaN  1.2680    NaN  NaN  NaN -0.173    NaN    NaN   
C3L-00098     NaN -0.4315 -0.0685    NaN  NaN  NaN -0.965  0.657    NaN   

Name               ... ZNF799  ZNF839 ZNF860  ZNRF2 ZRANB2         ZRSR2  \
Site         Y439  ...   Y430    Y283    Y49   Y111   Y114   Y124   Y347   
Patient_ID         ...                                                     
C3L-00006   0.312  ...    NaN     NaN    NaN    NaN    NaN  0.181  0.788   
C3L-00008     NaN  ...    NaN  0.0684    NaN    NaN    NaN -0.929    NaN   
C3L-00032     NaN  ...    NaN     NaN    NaN    NaN    NaN -0.283  0.675   
C3L-00090     NaN  ...    NaN     NaN    NaN    NaN    NaN -0.123 -0.026   
C3L-00098     NaN  ...    NaN     NaN    NaN -0.058    NaN  0.109    NaN   

Name          ZYX ZZEF1    ZZZ3  
Site         Y172 Y1795    Y399  
Patient_ID                       
C3L-00006  -0.378   NaN     NaN  
C3L-00008  -1.230   NaN -0.0635  
C3L-00032   0.700   NaN     NaN  
C3L-00090  -1.020   NaN  0.2980  
C3L-00098  -0.636   NaN -0.2900  

[5 rows x 2397 columns]

# How to use `cptac.utils.reduce_multiindex()`
To make it easier to work with multi-level indices, we provide the `reduce_multiindex` function, available for import from the `cptac.utils` submodule. It can both drop levels from a multiindex, and "flatten" a multi-level index into a single-level index by concatenating the keys from multiple levels into a single key for each column.

In [7]:
import cptac.utils as ut

## Dropping Levels
We can drop levels based on index or name. We can also drop single or multiple levels at once. 
Note that it will warn you if duplicate column key combinations arise due to dropping levels. 

### Dropping by index or name

In [8]:
ut.reduce_multiindex(df=phospho, levels_to_drop="Site").head()

cptac warning: Due to dropping the specified levels, dataframe now has 71331 duplicated column headers. (<ipython-input-8-760f4b3caa48>, line 1)


Name,AAAS,AAAS,AAAS,AACS,AAED1,AAGAB,AAGAB,AAK1,AAK1,AAK1,...,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,NaN,NaN,NaN,-0.881,-1.810,NaN,NaN,NaN,-0.242,-0.2420,...,0.18400,NaN,NaN,NaN,-0.20500,NaN,NaN,NaN,NaN,NaN
C3L-00008,NaN,NaN,NaN,NaN,0.084,NaN,NaN,-1.110,-0.383,-1.0900,...,-0.17100,NaN,NaN,-0.393,-0.17100,NaN,0.29,NaN,0.1605,-0.0635
C3L-00032,-0.202,NaN,NaN,NaN,-1.880,NaN,NaN,NaN,0.382,-0.0416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C3L-00090,-0.002,NaN,-0.407,NaN,NaN,NaN,NaN,NaN,NaN,-0.5550,...,0.13970,NaN,NaN,NaN,-0.55900,NaN,NaN,NaN,NaN,0.2980
C3L-00098,0.556,-0.0461,NaN,NaN,0.941,NaN,0.429,0.362,0.697,-0.0529,...,-0.15875,NaN,NaN,0.196,0.06175,NaN,NaN,NaN,NaN,-0.2900


### Dropping single or multiple levels at once
By passing a list (or array-like) to levels_to drop, we can drop multiple levels of the multiindex at the same time. Note that we must leave at least one existing level. 

We will show this with the colon data.

In [ ]:
colon = cptac.Colon()
phospho = colon.get_phosphoproteomics()
phospho.head()

In [ ]:
# Drop evey level except Database_ID
drop = ["Name", "Site"]
ut.reduce_multiindex(df=phospho, levels_to_drop=drop).head()

## Combining levels (Flattening)

We can combine levels of a multiindexed dataframe. When combined the levels will be sepereated by an underscore, by default. We could specify a different seperator using the `sep` parameter.

In [ ]:
ut.reduce_multiindex(df=phospho, flatten=True).head()

When flatteing levels , NaNs and empty strings will automitically be dropped.

In [12]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics")
phospho_and_CNV.head()

# Note that the CNV columns all have empty strings in the "Site" level of the columns,
# since the CNV data doesn't have any values for that.

cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-00006.N, C3L-00361.N, C3L-00563.N, C3L-00586.N, C3L-00601.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00932.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01282.N, C3L-01304.N, C3L-01307.N, C3L-01311.N, C3L-01744.N, C3N-00200.N, C3N-00333.N, C3N-00383.N, C3N-00729.N, C3N-00858.N, C3N-00866.N, C3N-01211.N, C3N-01346.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (<ipython-input-12-90868f8b8d93>, line 1)


Name       A1BG_CNV A1BG-AS1_CNV A1CF_CNV A2M_CNV A2M-AS1_CNV A2ML1_CNV  \
Site                                                                      
Patient_ID                                                                
C3L-00006      0.00         0.00    -0.01    0.00        0.00      0.00   
C3L-00008      0.01         0.01     0.00    0.00        0.00      0.00   
C3L-00032      0.01         0.01     0.00    0.00        0.00      0.00   
C3L-00090     -0.04        -0.04     0.36   -0.05       -0.05     -0.05   
C3L-00098      0.85         0.85     0.16   -0.41       -0.41     -0.41   

Name       A2MP1_CNV A3GALT2_CNV A4GALT_CNV A4GNT_CNV  ...  \
Site                                                   ...   
Patient_ID                                             ...   
C3L-00006       0.00        0.00       0.01     -0.01  ...   
C3L-00008       0.00        0.00       0.01      0.00  ...   
C3L-00032       0.00        0.01       0.00      0.00  ...   
C3L-00090      -0.05       -0.04      -0.04     -0.05  ...   
C3L-00098      -0.41       -0.12       0.15      0.19  ...   

Name       ZZZ3_phosphoproteomics                                            \
Site                         S397 S411 S420   S424     S426 S468   S89 T415   
Patient_ID                                                                    
C3L-00006                 0.18400  NaN  NaN    NaN -0.20500  NaN   NaN  NaN   
C3L-00008                -0.17100  NaN  NaN -0.393 -0.17100  NaN  0.29  NaN   
C3L-00032                     NaN  NaN  NaN    NaN      NaN  NaN   NaN  NaN   
C3L-00090                 0.13970  NaN  NaN    NaN -0.55900  NaN   NaN  NaN   
C3L-00098                -0.15875  NaN  NaN  0.196  0.06175  NaN   NaN  NaN   

Name                        
Site          T418    Y399  
Patient_ID                  
C3L-00006      NaN     NaN  
C3L-00008   0.1605 -0.0635  
C3L-00032      NaN     NaN  
C3L-00090      NaN  0.2980  
C3L-00098      NaN -0.2900  

[5 rows x 101269 columns]

In [13]:
ut.reduce_multiindex(df=phospho_and_CNV, flatten=True).head()
# Notice that the empty strings have been dropped

Name,A1BG_CNV,A1BG-AS1_CNV,A1CF_CNV,A2M_CNV,A2M-AS1_CNV,A2ML1_CNV,A2MP1_CNV,A3GALT2_CNV,A4GALT_CNV,A4GNT_CNV,...,ZZZ3_phosphoproteomics_S397,ZZZ3_phosphoproteomics_S411,ZZZ3_phosphoproteomics_S420,ZZZ3_phosphoproteomics_S424,ZZZ3_phosphoproteomics_S426,ZZZ3_phosphoproteomics_S468,ZZZ3_phosphoproteomics_S89,ZZZ3_phosphoproteomics_T415,ZZZ3_phosphoproteomics_T418,ZZZ3_phosphoproteomics_Y399
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,0.00,0.00,-0.01,0.00,0.00,0.00,0.00,0.00,0.01,-0.01,...,0.18400,NaN,NaN,NaN,-0.20500,NaN,NaN,NaN,NaN,NaN
C3L-00008,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,...,-0.17100,NaN,NaN,-0.393,-0.17100,NaN,0.29,NaN,0.1605,-0.0635
C3L-00032,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C3L-00090,-0.04,-0.04,0.36,-0.05,-0.05,-0.05,-0.05,-0.04,-0.04,-0.05,...,0.13970,NaN,NaN,NaN,-0.55900,NaN,NaN,NaN,NaN,0.2980
C3L-00098,0.85,0.85,0.16,-0.41,-0.41,-0.41,-0.41,-0.12,0.15,0.19,...,-0.15875,NaN,NaN,0.196,0.06175,NaN,NaN,NaN,NaN,-0.2900


## Getting a single level index of tuples

You can also use `reduce_multiindex` to turn the multi-level column index into a single level index of tuples, with each value in a column's tuple corresponding to the column's value for that level of the index:

In [14]:
ut.reduce_multiindex(df=phospho, tuples=True).head()

,"(AAAS, S495, Q9NRG9)","(AAAS, S525, Q9NRG9)","(AAAS, S541, Q9NRG9)","(AAED1, S12, Q7RTV5)","(AAGAB, S310, Q6PD74)","(AAGAB, S311, Q6PD74)","(AAK1, S20, Q2M2I8)","(AAK1, S21, Q2M2I8)","(AAK1, S26, Q2M2I8)","(AAK1, S618, Q2M2I8)",...,"(ZZEF1, S1501, O43149)","(ZZEF1, S1518, O43149)","(ZZEF1, S1537, O43149)","(ZZEF1, S1540, O43149)","(ZZEF1, T1521, O43149)","(ZZZ3, S113, Q8IYH5)","(ZZZ3, S391, Q8IYH5)","(ZZZ3, S606, Q8IYH5)","(ZZZ3, S90, Q8IYH5)","(ZZZ3, S91, Q8IYH5)"
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01CO005,NaN,NaN,-0.240,-0.460,NaN,NaN,-0.231,NaN,NaN,NaN,...,NaN,-0.6750,-1.404,-1.404,NaN,-0.572,NaN,0.205,NaN,NaN
01CO006,-0.3650,NaN,NaN,-0.424,-0.015,-0.015,NaN,-0.485,NaN,NaN,...,NaN,-0.2875,0.222,0.222,-0.701,0.624,NaN,NaN,NaN,NaN
01CO008,0.7250,0.137,0.137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.147,-0.1470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03
01CO013,0.2265,NaN,NaN,-1.278,0.403,0.075,-0.223,-0.701,NaN,NaN,...,-0.041,0.0430,0.554,0.554,0.127,1.263,NaN,NaN,NaN,NaN
01CO014,0.5600,NaN,NaN,-0.382,NaN,NaN,NaN,-0.259,NaN,NaN,...,NaN,-0.1290,-0.919,-0.919,NaN,0.032,NaN,0.026,NaN,NaN


## Turning off warnings

If your multiindex operation creates duplicate column headers, or has no effect, `reduce_multiindex` will warn you. You can silence these warnings by passing `True` to the `quiet` parameter:

In [15]:
ut.reduce_multiindex(df=phospho, levels_to_drop=["Name", "Database_ID"]).head()

cptac warning: Due to dropping the specified levels, dataframe now has 29529 duplicated column headers. (<ipython-input-15-fb45a8052997>, line 1)


Site,S495,S525,S541,S12,S310,S311,S20,S21,S26,S618,...,S1501,S1518,S1537,S1540,T1521,S113,S391,S606,S90,S91
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01CO005,NaN,NaN,-0.240,-0.460,NaN,NaN,-0.231,NaN,NaN,NaN,...,NaN,-0.6750,-1.404,-1.404,NaN,-0.572,NaN,0.205,NaN,NaN
01CO006,-0.3650,NaN,NaN,-0.424,-0.015,-0.015,NaN,-0.485,NaN,NaN,...,NaN,-0.2875,0.222,0.222,-0.701,0.624,NaN,NaN,NaN,NaN
01CO008,0.7250,0.137,0.137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.147,-0.1470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03
01CO013,0.2265,NaN,NaN,-1.278,0.403,0.075,-0.223,-0.701,NaN,NaN,...,-0.041,0.0430,0.554,0.554,0.127,1.263,NaN,NaN,NaN,NaN
01CO014,0.5600,NaN,NaN,-0.382,NaN,NaN,NaN,-0.259,NaN,NaN,...,NaN,-0.1290,-0.919,-0.919,NaN,0.032,NaN,0.026,NaN,NaN


In [16]:
# No warning will be issued
ut.reduce_multiindex(df=phospho, levels_to_drop=["Name", "Database_ID"], quiet=True).head()

Site,S495,S525,S541,S12,S310,S311,S20,S21,S26,S618,...,S1501,S1518,S1537,S1540,T1521,S113,S391,S606,S90,S91
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01CO005,NaN,NaN,-0.240,-0.460,NaN,NaN,-0.231,NaN,NaN,NaN,...,NaN,-0.6750,-1.404,-1.404,NaN,-0.572,NaN,0.205,NaN,NaN
01CO006,-0.3650,NaN,NaN,-0.424,-0.015,-0.015,NaN,-0.485,NaN,NaN,...,NaN,-0.2875,0.222,0.222,-0.701,0.624,NaN,NaN,NaN,NaN
01CO008,0.7250,0.137,0.137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.147,-0.1470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03
01CO013,0.2265,NaN,NaN,-1.278,0.403,0.075,-0.223,-0.701,NaN,NaN,...,-0.041,0.0430,0.554,0.554,0.127,1.263,NaN,NaN,NaN,NaN
01CO014,0.5600,NaN,NaN,-0.382,NaN,NaN,NaN,-0.259,NaN,NaN,...,NaN,-0.1290,-0.919,-0.919,NaN,0.032,NaN,0.026,NaN,NaN
